In [17]:
#basics
import random
import pandas as pd
import torch
#extra:
from lxml import etree
from glob import glob

In [98]:
def parse_data(data_dir):
        #sentence_id : id for the sentence, can be found in the xml data
        #token_id: id for the word in the vocabulary
        #char_start_id: where the character starts in the sentence, can be found in the xml data
        #char_end_id: where the character ends in the sentence, can be found in the xml data
        #split: which data split the token belong to, e.g. TRAIN, VAL or TEST.
        #ner_id: id of the NER label, e.g. if we have 3 labels there would be 3 ids.
        test = []
        train = []
        vocabulary = []
        
        #print(os.listdir(os.path.expanduser(data_dir)))
        data_dir = glob("{}/*".format(data_dir))
        
        for subdir in data_dir:
            #print("subdir: ", subdir)
            subdir = glob("{}/*".format(subdir))
            for folder in subdir:
                #print("folder_pre_glob: ", folder)
                folder = glob("{}/*".format(folder))
                #print("folder_after_glob: ", folder)
                for subfolder in folder:
                    #print("subfolder: ", subfolder)
                    subfolder = glob("{}/*".format(subfolder))
                    #print(len(subfolder))
                    for xml_file in subfolder:
                        print("XML_FILE: ", xml_file)
                        tree = etree.parse(xml_file)
                        print("TREE: ", tree)
                        root = etree.tostring(tree.getroot())
                        #xml_file = glob("{}/*".format(xml_file))
                        print("ROOT: ", root)
                        print("##################################################################")
    

In [99]:
parse_data('/home/guserbto@GU.GU.SE/lt2316-h20-aa/DDICorpus')

XML_FILE:  /home/guserbto@GU.GU.SE/lt2316-h20-aa/DDICorpus/Test/Test for DDI Extraction task/DrugBank/Abacavir.xml
TREE:  <lxml.etree._ElementTree object at 0x7fada277da00>
ROOT:  b'<document id="DDI-DrugBank.d610">\n    <sentence id="DDI-DrugBank.d610.s0" text="Pharmacokinetic properties of abacavir were not altered by the addition of either lamivudine or zidovudine or the combination of lamivudine and zidovudine. ">\n        <entity id="DDI-DrugBank.d610.s0.e0" charOffset="30-37" type="drug" text="abacavir"/>\n        <entity id="DDI-DrugBank.d610.s0.e1" charOffset="82-91" type="drug" text="lamivudine"/>\n        <entity id="DDI-DrugBank.d610.s0.e2" charOffset="96-105" type="drug" text="zidovudine"/>\n        <entity id="DDI-DrugBank.d610.s0.e3" charOffset="129-138" type="drug" text="lamivudine"/>\n        <entity id="DDI-DrugBank.d610.s0.e4" charOffset="144-153" type="drug" text="zidovudine"/>\n        <pair id="DDI-DrugBank.d610.s0.p0" e1="DDI-DrugBank.d610.s0.e0" e2="DDI-DrugBank.

ROOT:  b'<document id="DDI-DrugBank.d676">\n    <sentence id="DDI-DrugBank.d676.s0" text="To minimize CNS depression and possible potentiation, barbiturates, antihistamines, narcotics, hypotensive agents or phenothiazines should be used with caution. ">\n        <entity id="DDI-DrugBank.d676.s0.e0" charOffset="54-65" type="group" text="barbiturates"/>\n        <entity id="DDI-DrugBank.d676.s0.e1" charOffset="68-81" type="group" text="antihistamines"/>\n        <entity id="DDI-DrugBank.d676.s0.e2" charOffset="84-92" type="group" text="narcotics"/>\n        <entity id="DDI-DrugBank.d676.s0.e3" charOffset="95-112" type="group" text="hypotensive agents"/>\n        <entity id="DDI-DrugBank.d676.s0.e4" charOffset="117-130" type="group" text="phenothiazines"/>\n        <pair id="DDI-DrugBank.d676.s0.p0" e1="DDI-DrugBank.d676.s0.e0" e2="DDI-DrugBank.d676.s0.e1" ddi="false"/>\n        <pair id="DDI-DrugBank.d676.s0.p1" e1="DDI-DrugBank.d676.s0.e0" e2="DDI-DrugBank.d676.s0.e2" ddi="false"/>\n   

ROOT:  b'<document id="DDI-MedLine.d154">\n    <sentence id="DDI-MedLine.d154.s0" text="Enhancement of humoral immune responses to inactivated Newcastle disease and avian influenza vaccines by oral administration of ginseng stem-and-leaf saponins in chickens.&#13;&#10;">\n        <entity id="DDI-MedLine.d154.s0.e0" charOffset="77-100" type="group" text="avian influenza vaccines"/>\n        <entity id="DDI-MedLine.d154.s0.e1" charOffset="128-157" type="drug_n" text="ginseng stem-and-leaf saponins"/>\n        <pair id="DDI-MedLine.d154.s0.p0" e1="DDI-MedLine.d154.s0.e0" e2="DDI-MedLine.d154.s0.e1" ddi="false"/>\n    </sentence>\n    <sentence id="DDI-MedLine.d154.s1" text="Newcastle disease (ND) and avian influenza (AI) are common in the poultry industry. "/>\n    <sentence id="DDI-MedLine.d154.s2" text="The objective of this study was to evaluate the effect of oral administration of ginseng stem-and-leaf saponins (GSLS) on the humoral immune responses of chickens to inactivated ND and A

TREE:  <lxml.etree._ElementTree object at 0x7fada278e960>
ROOT:  b'<document id="DDI-MedLine.d145">\n    <sentence id="DDI-MedLine.d145.s0" text="IGF1R-targeted therapy and its enhancement of doxorubicin chemosensitivity in human osteosarcoma cell lines.&#13;&#10;">\n        <entity id="DDI-MedLine.d145.s0.e0" charOffset="46-56" type="drug" text="doxorubicin"/>\n    </sentence>\n    <sentence id="DDI-MedLine.d145.s1" text="Type-I insulin-like growth factor receptor (IGF1R) and its signaling play an important role in osteosarcomagenesis, tumor progression, and chemoresistance. "/>\n    <sentence id="DDI-MedLine.d145.s2" text="The purpose of this study was to investigate both the effect and mechanisms of IGF1R inhibition by tyrphostin AG1024 in the presence or absence of doxorubicin in a panel of six osteosarcoma cell lines and a self-established doxorubicin-resistant cell line. ">\n        <entity id="DDI-MedLine.d145.s2.e0" charOffset="99-115" type="drug_n" text="tyrphostin AG1024"/>\n

In [51]:
xml = '<a xmlns="test"><b xmlns="test"/></a>'
root = etree.fromstring(xml)
etree.tostring(root)

b'<a xmlns="test"><b xmlns="test"/></a>'